In [29]:
# Создаем таблицы и загружаем данные 
import io
import csv
import psycopg2

conn = psycopg2.connect(
    dbname="shod_1",
    user="dmp",
    password="dmp",
    host="localhost",
    port="5434"
)

conn.autocommit = True
cursor = conn.cursor()

create_customer_table = """
CREATE TABLE IF NOT EXISTS customer(
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(30),
    dob VARCHAR(50),
    job_title VARCHAR(50),
    job_industry_category VARCHAR(50),
    wealth_segment VARCHAR(50),
    deceased_indicator VARCHAR(50),
    owns_car VARCHAR(30),
    address VARCHAR(50),
    postcode VARCHAR(30),
    state VARCHAR(30),
    country VARCHAR(30),
    property_valuation INT
);
"""
cursor.execute(create_customer_table)

create_transaction_table = """
CREATE TABLE IF NOT EXISTS "transaction" (
    transaction_id INT PRIMARY KEY,
    product_id INT,
    customer_id INT,
    transaction_date VARCHAR(30),
    online_order VARCHAR(30),
    order_status VARCHAR(30),
    brand VARCHAR(30),
    product_line VARCHAR(30),
    product_class VARCHAR(30),
    product_size VARCHAR(30),
    list_price FLOAT4,
    standard_cost FLOAT4
);
"""
cursor.execute(create_transaction_table)

with open('customer.csv', 'r', encoding='utf-8') as f:
    cursor.copy_expert("COPY customer FROM STDIN WITH CSV HEADER DELIMITER ';'", f)

with open('transaction.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        header = next(reader)
        processed_rows = []
        for row in reader:
            row[10] = row[10].replace(',', '.')
            row[11] = row[11].replace(',', '.')
            processed_rows.append(row)
    
output = io.StringIO()
writer = csv.writer(output, delimiter=';')
writer.writerow(header)
writer.writerows(processed_rows)
output.seek(0)
    
cursor.copy_expert('COPY "transaction" FROM STDIN WITH CSV HEADER DELIMITER \';\'', output)
    


In [30]:
# Задание 1

sql_query = """
SELECT job_industry_category, COUNT(*) AS client_count
FROM customer
GROUP BY job_industry_category
ORDER BY client_count DESC;
"""
cursor.execute(sql_query)
results = cursor.fetchall()

for row in results:
    print(row)

('Manufacturing', 799)
('Financial Services', 774)
('n/a', 656)
('Health', 602)
('Retail', 358)
('Property', 267)
('IT', 223)
('Entertainment', 136)
('Argiculture', 113)
('Telecommunications', 72)


In [31]:
# Задание 2

sql_query = """
SELECT DATE_TRUNC('month', TO_DATE(t.transaction_date, 'DD-MM-YYYY')) AS month,
       c.job_industry_category,
       SUM(t.list_price) AS total_transactions
FROM "transaction" t
JOIN customer c ON t.customer_id = c.customer_id
GROUP BY month, c.job_industry_category
ORDER BY month, c.job_industry_category;
"""
cursor.execute(sql_query)
results = cursor.fetchall()

for row in results:
    print(row)

(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Argiculture', 43513.83)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Entertainment', 64089.918)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Financial Services', 366383.7)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Health', 286860.4)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'IT', 107783.38)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Manufacturing', 365232.38)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Property', 100686.96)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Retail', 182375.69)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Telecommunications', 31210.201)
(datetime.datetime(2017, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'n/a', 316819.78)
(datetime.datetime(2017, 2, 1, 0, 0, tzinfo=datetime.timezone.utc), 'Argicu

In [32]:
# Задание 3

sql_query = """
SELECT t.brand, COUNT(*) AS online_order_count
FROM "transaction" t
JOIN customer c ON t.customer_id = c.customer_id
WHERE t.order_status = 'Approved'
  AND c.job_industry_category = 'IT'
  AND t.online_order = 'True'
GROUP BY t.brand
ORDER BY t.brand;
"""
cursor.execute(sql_query)
results = cursor.fetchall()

for row in results:
    print(row)

('Giant Bicycles', 89)
('Norco Bicycles', 92)
('OHM Cycles', 78)
('Solex', 101)
('Trek Bicycles', 82)
('WeareA2B', 90)
(None, 8)


In [33]:
# Задание 4

# Выполнение запроса с использованием GROUP BY
sql_query_group_by = """
SELECT customer_id,
       SUM(list_price) AS total_amount,
       MAX(list_price) AS max_amount,
       MIN(list_price) AS min_amount,
       COUNT(*) AS transaction_count
FROM "transaction"
GROUP BY customer_id;
"""
cursor.execute(sql_query_group_by)
results_group_by = cursor.fetchall()

# Выполнение запроса с использованием оконных функций
sql_query_window = """
SELECT DISTINCT customer_id,
       SUM(list_price) OVER (PARTITION BY customer_id) AS total_amount,
       MAX(list_price) OVER (PARTITION BY customer_id) AS max_amount,
       MIN(list_price) OVER (PARTITION BY customer_id) AS min_amount,
       COUNT(*) OVER (PARTITION BY customer_id) AS transaction_count
FROM "transaction";
"""
cursor.execute(sql_query_window)
results_window = cursor.fetchall()

# Преобразуем результаты в словари для удобного сравнения (ключ - customer_id)
group_by_dict = {row[0]: row[1:] for row in results_group_by}
window_dict = {row[0]: row[1:] for row in results_window}

assert len(set(group_by_dict.keys()).difference(set(window_dict.keys()))) == 0

for customer_id in group_by_dict:
    if group_by_dict[customer_id] != window_dict[customer_id]:
        print(f"Различия {customer_id}: GROUP BY: {group_by_dict[customer_id]}, WINDOW: {window_dict[customer_id]}")


Различия 273: GROUP BY: (11372.24, 1992.93, 100.35, 10), WINDOW: (11372.239, 1992.93, 100.35, 10)
Различия 2614: GROUP BY: (6322.4097, 1894.19, 1198.46, 4), WINDOW: (6322.41, 1894.19, 1198.46, 4)
Различия 1750: GROUP BY: (7768.6396, 1894.19, 360.4, 6), WINDOW: (7768.64, 1894.19, 360.4, 6)
Различия 1003: GROUP BY: (9111.19, 1769.64, 60.34, 9), WINDOW: (9111.189, 1769.64, 60.34, 9)
Различия 292: GROUP BY: (8825.971, 1977.36, 441.49, 7), WINDOW: (8825.97, 1977.36, 441.49, 7)
Различия 1331: GROUP BY: (11641.262, 1977.36, 183.86, 10), WINDOW: (11641.26, 1977.36, 183.86, 10)
Различия 2784: GROUP BY: (2423.2202, 1577.53, 71.16, 3), WINDOW: (2423.22, 1577.53, 71.16, 3)
Различия 1552: GROUP BY: (6945.16, 1992.93, 945.04, 5), WINDOW: (6945.1597, 1992.93, 945.04, 5)
Различия 1811: GROUP BY: (10112.53, 1769.64, 499.53, 8), WINDOW: (10112.529, 1769.64, 499.53, 8)
Различия 113: GROUP BY: (5412.97, 2083.94, 363.01, 6), WINDOW: (5412.9697, 2083.94, 363.01, 6)
Различия 2172: GROUP BY: (6844.5, 2091.47,

Как можно заметить результат не совпадает на 100% из-за различного округления разных способах

In [34]:
# Задание 5

# Запрос для получения клиентов с минимальной суммой транзакций
sql_query_min = """
WITH customer_totals AS (
    SELECT customer_id, SUM(list_price) AS total_amount
    FROM "transaction"
    GROUP BY customer_id
)
SELECT c.first_name, c.last_name, ct.total_amount
FROM customer_totals ct
JOIN customer c ON ct.customer_id = c.customer_id
WHERE ct.total_amount = (SELECT MIN(total_amount) FROM customer_totals)
  AND ct.total_amount IS NOT NULL;
"""
cursor.execute(sql_query_min)
results_min = cursor.fetchall()

print("Клиенты с минимальной суммой транзакций:")
for row in results_min:
    print(row)


# Запрос для получения клиентов с максимальной суммой транзакций
sql_query_max = """
WITH customer_totals AS (
    SELECT customer_id, SUM(list_price) AS total_amount
    FROM "transaction"
    GROUP BY customer_id
)
SELECT c.first_name, c.last_name, ct.total_amount
FROM customer_totals ct
JOIN customer c ON ct.customer_id = c.customer_id
WHERE ct.total_amount = (SELECT MAX(total_amount) FROM customer_totals)
  AND ct.total_amount IS NOT NULL;
"""
cursor.execute(sql_query_max)
results_max = cursor.fetchall()

print("\nКлиенты с максимальной суммой транзакций:")
for row in results_max:
    print(row)


Клиенты с минимальной суммой транзакций:
('Hamlen', 'Slograve', 60.34)

Клиенты с максимальной суммой транзакций:
('Jillie', 'Fyndon', 19071.322)


In [35]:
# Задание 6

sql_query = """
SELECT *
FROM (
  SELECT t.*,
         ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY TO_DATE(transaction_date, 'DD.MM.YYYY')) AS rn
  FROM "transaction" t
) sub
WHERE rn = 1;
"""
cursor.execute(sql_query)
results = cursor.fetchall()

for row in results:
    print(row)


(9785, 72, 1, '05.01.2017', 'False', 'Approved', 'Norco Bicycles', 'Standard', 'medium', 'medium', 360.4, 270.3, 1)
(2261, 1, 2, '04.05.2017', 'True', 'Approved', 'Giant Bicycles', 'Standard', 'medium', 'medium', 1403.5, 954.82, 1)
(10302, 33, 3, '23.02.2017', 'False', 'Approved', 'Giant Bicycles', 'Standard', 'medium', 'small', 1311.44, 1167.18, 1)
(12441, 95, 4, '03.04.2017', 'False', 'Approved', 'Giant Bicycles', 'Standard', 'medium', 'large', 569.56, 528.43, 1)
(2291, 23, 5, '03.03.2017', 'True', 'Approved', 'Norco Bicycles', 'Mountain', 'low', 'small', 688.63, 612.88, 1)
(7096, 77, 6, '28.01.2017', 'False', 'Approved', 'WeareA2B', 'Standard', 'medium', 'medium', 1769.64, 108.76, 1)
(18369, 72, 7, '18.02.2017', 'True', 'Approved', 'Norco Bicycles', 'Standard', 'medium', 'medium', 360.4, 270.3, 1)
(10792, 89, 8, '04.01.2017', 'True', 'Approved', 'WeareA2B', 'Touring', 'medium', 'large', 1362.99, 57.74, 1)
(8591, 69, 9, '04.02.2017', 'False', 'Approved', 'Giant Bicycles', 'Road', 'me

In [36]:
# Задание 7

sql_query = """
WITH transaction_intervals AS (
  SELECT customer_id,
         TO_DATE(transaction_date, 'DD.MM.YYYY') AS trans_date,
         LAG(TO_DATE(transaction_date, 'DD.MM.YYYY')) OVER (PARTITION BY customer_id ORDER BY TO_DATE(transaction_date, 'DD.MM.YYYY')) AS prev_date
  FROM "transaction"
),
intervals AS (
  SELECT customer_id, (trans_date - prev_date) AS gap
  FROM transaction_intervals
  WHERE prev_date IS NOT NULL
),
max_gap AS (
  SELECT MAX(gap) AS max_gap_value FROM intervals
)
SELECT DISTINCT c.first_name, c.last_name, c.job_title
FROM intervals i
JOIN max_gap mg ON i.gap = mg.max_gap_value
JOIN customer c ON i.customer_id = c.customer_id;
"""
cursor.execute(sql_query)
results = cursor.fetchall()

for row in results:
    print(row)


('Susanetta', None, 'Legal Assistant')
